In [17]:
# Dependencies
import pandas as pd
from datetime import datetime

In [3]:
# read csv file into dataframe
csv_file = "../Resources/weather_data_nyc_centralpark_2016.csv"
df_weather_data = pd.read_csv(csv_file)
df_weather_data.head()

,date,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,1-1-2016,42,34,38.0,0.00,0.0,0
1,2-1-2016,40,32,36.0,0.00,0.0,0
2,3-1-2016,45,35,40.0,0.00,0.0,0
3,4-1-2016,36,14,25.0,0.00,0.0,0
4,5-1-2016,29,11,20.0,0.00,0.0,0


In [8]:
df_weather_data["precipitation"] = df_weather_data["precipitation"].replace('T', '', regex=True)
df_weather_data["snow fall"] = df_weather_data["snow fall"].replace('T', '', regex=True)
df_weather_data["snow depth"] = df_weather_data["snow depth"].replace('T', '', regex=True)

In [13]:
def try_parsing_date(text):
    for fmt in ('%m/%d/%Y', '%d-%m-%Y'):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            pass
    raise ValueError('no valid date format found')

In [20]:
i = 0
for date in df_weather_data["date"]:
    df_weather_data["date"][i] = try_parsing_date(date)
    i += 1

C:\Users\Allen\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
df_weather_data.head()

,date,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,2016-01-01 00:00:00,42,34,38.0,0.00,0.0,0
1,2016-01-02 00:00:00,40,32,36.0,0.00,0.0,0
2,2016-01-03 00:00:00,45,35,40.0,0.00,0.0,0
3,2016-01-04 00:00:00,36,14,25.0,0.00,0.0,0
4,2016-01-05 00:00:00,29,11,20.0,0.00,0.0,0


In [23]:
df_weather_data.to_csv("../Resources/Cleaned_Weather_Data.csv", index=False)